In [1]:
%pip install progressbar2
%pip install gym

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827621 sha256=199be7e578489f6a6699c8264776071e31c73811a365c614cb3320ae7170c535
  Stored in directory: /home/halozhan/.cache/pip/wheels/17/79/65/7afedc162d858b02708a3b8f7a6dd5b1000dcd5b0f894f7cc1
Successfully built gym
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 코드 12-1 라이브러리 호출
import numpy as np
import random
from IPython.display import clear_output
from collections import deque  # 에이전트가 환경에 반응한 경험을 저장
import gym
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam

2024-12-03 15:13:04.774146: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-12-03 15:13:04.774187: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
%pip install gym[toy_text]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 코드 12-2 ‘Taxi-v3’에 대한 환경
# Taxi-v3 객체를 인스턴스로 생성
env = gym.make("Taxi-v3", render_mode="ansi")
env.reset()

# 환경 및 에이전트의 현재 상태를 render 메서드를 사용하여 표시
print(env.render())

print('취할 수 있는 상태 수: {}'.format(env.observation_space.n)) # 환경의 모든 상태에 접근
print('취할 수 있는 행동 수: {}'.format(env.action_space.n)) # 환경의 모든 행동에 접근

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


취할 수 있는 상태 수: 500
취할 수 있는 행동 수: 6


In [4]:
# 코드 12-3 에이전트 구현
class Agent:
    def __init__(self, env, optimizer):  # 상태와 행동을 초기화
        self._state_size = (
            env.observation_space.n
        )  # 환경 속성 observation_space를 초기화
        self._action_size = env.action_space.n  # 환경 속성 action_space를 초기화
        self._optimizer = optimizer  # 옵티마이저 초기화
        self.expirience_replay = deque(maxlen=2000)  # 과거 행동에 대한 기억을 초기화
        self.gamma = 0.6  # 할인율 초기화
        self.epsilon = 0.1  # 탐험 비율 초기화

        self.q_network = (
            self.build_compile()
        )  # build_compile() 함수를 사용하여 큐-네트워크 구성
        self.target_network = (
            self.build_compile()
        )  # build_compile() 함수를 사용하여 타깃 큐-네트워크 구성
        self.target_model()

    def store(self, state, action, reward, next_state, terminated):
        self.expirience_replay.append((state, action, reward, next_state, terminated))

    def build_compile(self):  # 네트워크 구성을 위한 함수
        model = Sequential()
        model.add(Embedding(self._state_size, 10, input_length=1))
        model.add(Reshape((10,)))
        model.add(Dense(50, activation="relu"))
        model.add(Dense(50, activation="relu"))
        model.add(Dense(50, activation="relu"))
        model.add(Dense(self._action_size, activation="linear"))
        model.compile(loss="mse", optimizer=self._optimizer)
        return model

    def target_model(self):  # 가중치를 적용하기 위한 함수
        self.target_network.set_weights(self.q_network.get_weights())

    def act(self, state):  # 탐험을 위한 함수
        if np.random.rand() <= self.epsilon:
            return env.action_space.sample()
        q_values = self.q_network.predict(state)
        return np.argmax(q_values[0])

    def retrain(self, batch_size):  # 큐-네트워크 훈련에 대한 함수
        # 리플레이 메모리에서 랜덤한 데이터 선택
        minibatch = random.sample(self.expirience_replay, batch_size)
        target = self.q_network.predict(state)
        if terminated:
            target[0][action] = reward
        else:
            t = self.target_network.predict(next_state)
            target[0][action] = reward + self.gamma * np.amax(t)
        self.q_network.fit(state, target, epochs=1, verbose=0)  # 큐-네트워크 훈련

In [5]:
# 코드 12-4 훈련 준비
optimizer = Adam(learning_rate=0.01)
agent = Agent(env, optimizer)
batch_size = 32
num_of_episodes = 10
timesteps_per_episode = 10
agent.q_network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             5000      
_________________________________________________________________
reshape (Reshape)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                550       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 306       
Total params: 10,956
Trainable params: 10,956
Non-trainable params: 0
____________________________________________________

2024-12-03 15:13:18.990366: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2024-12-03 15:13:19.152847: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:968] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-03 15:13:19.152876: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.759GHz coreCount: 20 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 298.32GiB/s
2024-12-03 15:13:19.152943: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-12-03 15:13:19.152988: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic 

In [17]:
# 코드 12-5 모델 훈련
for e in range(0, num_of_episodes):
    state = env.reset()  # 환경 재설정
    print(f"Initial state: {state}")  # state의 구조 확인
    state = (
        np.array([state[0]]).reshape([1, -1]).astype(np.float32)
    )  # 첫 번째 요소만 사용하여 배열로 변환

    reward = 0  # 보상 변수 초기화
    terminated = False

    for timestep in range(timesteps_per_episode):
        action = agent.act(state)  # act() 함수 실행

        # 에이전트가 단계별 행동을 취합니다.
        result = env.step(action)
        print(f"Step result: {result}")  # step 결과의 구조 확인
        next_state, reward, terminated, truncated, info = result
        print(f"Next state: {next_state}")  # next_state의 구조 확인
        # 첫 번째 요소만 사용하여 배열로 변환
        next_state = np.array([next_state]).reshape([1, -1]).astype(np.float32)
        agent.store(state, action, reward, next_state, terminated)
        state = next_state

        if terminated:
            agent.target_model()
            break

        if len(agent.expirience_replay) > batch_size:
            agent.retrain(batch_size)

    if (e + 1) % 10 == 0:
        print("**********************************")
        print("Episode: {}".format(e + 1))
        print(env.render())
        print("**********************************")

Initial state: (228, {'prob': 1.0, 'action_mask': array([1, 1, 1, 1, 0, 0], dtype=int8)})
Step result: (328, -1, False, False, {'prob': 1.0, 'action_mask': array([1, 1, 1, 0, 0, 0], dtype=int8)})
Next state: 328
Step result: (428, -1, False, False, {'prob': 1.0, 'action_mask': array([0, 1, 1, 0, 0, 0], dtype=int8)})
Next state: 428
Step result: (428, -1, False, False, {'prob': 1.0, 'action_mask': array([0, 1, 1, 0, 0, 0], dtype=int8)})
Next state: 428
Step result: (428, -1, False, False, {'prob': 1.0, 'action_mask': array([0, 1, 1, 0, 0, 0], dtype=int8)})
Next state: 428
Step result: (428, -1, False, False, {'prob': 1.0, 'action_mask': array([0, 1, 1, 0, 0, 0], dtype=int8)})
Next state: 428
Step result: (428, -1, False, False, {'prob': 1.0, 'action_mask': array([0, 1, 1, 0, 0, 0], dtype=int8)})
Next state: 428
Step result: (428, -1, False, False, {'prob': 1.0, 'action_mask': array([0, 1, 1, 0, 0, 0], dtype=int8)})
Next state: 428
Step result: (428, -1, False, False, {'prob': 1.0, 'acti